In [ ]:
import csv

tsv_file = open("trans.tsv")
txt_file = open("trans.txt", "w")

read_tsv = csv.reader(tsv_file, delimiter="\t")

for row in read_tsv:
  joined_string = "/t".join(row)
  txt_file.writelines(joined_string+ '\n')

txt_file.close()

In [ ]:
#with datafrane
import pandas as pd
colnames = ['src-tag', 'source', 'tar-tag', 'target']
data_df = pd.read_csv('/content/trans.tsv', sep='\t', names=colnames)

data_df = data_df[['source', 'target']]
source_sentences = []
target_sentences = []

for index, row in data_df.iterrows():
  src, tar = row[0], row[1]
  source_sentences.append(src)
  target_sentences.append(tar)
#print(source_sentences[0], len(source_sentences))
#print(target_sentences[0], len(target_sentences))
  

  


Let's try something. 1746
چلو کچھ نہ کچھ کرنے کی کوشش کرتے ہیں۔ 1746


In [ ]:
import re 
import fileinput

with open('trans.tsv', 'r') as f1, open('trans1.txt', 'w') as f2:
    f2.write("".join([c for c in f1.read() if not c.isdigit()]))
with open('trans1.txt', 'r') as f2, open('trans2.txt', 'w') as f3:
    for line in f2:
     new_line = (line.replace('\t', ' '))
     f3.write(new_line) 
f2.close()
f3.close()   

In [ ]:
import string
import numpy as np

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Model
from keras.layers import LSTM, Input, TimeDistributed, Dense, Activation, RepeatVector, Embedding
from tensorflow.keras.optimizers import Adam
from keras.losses import sparse_categorical_crossentropy

# Path to translation file
path_to_data = '/content/trans2.txt'

# Read file
translation_file = open(path_to_data,"r", encoding='utf-8') 
raw_data = translation_file.read()
translation_file.close()

# Parse data
raw_data = raw_data.split('\n')
pairs = [sentence.split('.') for sentence in  raw_data]
#pairs_test = pairs[:1000]
#pairs_train = pairs[1000:]
#pairs = pairs[1000:20000]
#print(pairs_test[0][0])
#print(pairs_train[0])

##Zainab Bongian

##Zainab Bonguan End

In [ ]:
def clean_sentence(sentence):
    # Lower case the sentence
    lower_case_sent = sentence.lower()
    # Strip punctuation
    string_punctuation = string.punctuation + "¡" + '¿'
    clean_sentence = lower_case_sent.translate(str.maketrans('', '', 
                                                        string_punctuation))
   
    return clean_sentence

In [ ]:
def tokenize(sentences):
    # Create tokenizer
    text_tokenizer = Tokenizer()
    # Fit texts
    text_tokenizer.fit_on_texts(sentences)
    return text_tokenizer.texts_to_sequences(sentences), text_tokenizer

In [ ]:
for pair in pairs:
  if len(pair)!=2:    
    print(pair[0])
  else:
    print(pair)
  break

[" You shouldn't say that kind of thing when children are around", '  آپ کو اس قسم کی باتیں بچوں کے سامنے نہیں کرنی چائیے۔']


In [ ]:
print(pair[0])

 You shouldn't say that kind of thing when children are around


In [ ]:
# Clean sentences
#english_sentences = [clean_sentence(pair) for pair in source_sentences]
#urdu_sentences = [clean_sentence(peir) for peir in target_sentences]

english_sentences = source_sentences
urdu_sentences = target_sentences

# Tokenize words
urd_text_tokenized, urd_text_tokenizer = tokenize(urdu_sentences)
eng_text_tokenized, eng_text_tokenizer = tokenize(english_sentences)

print('Maximum length urdu sentence: {}'.format(len(max(urd_text_tokenized,
                                                        key=len))))
print('Maximum length english sentence: {}'.format(len(max(eng_text_tokenized,
                                                           key=len))))


# Check language length
urdu_vocab = len(urd_text_tokenizer.word_index) + 1
english_vocab = len(eng_text_tokenizer.word_index) + 1
print("urdu vocabulary is of {} unique words".format(urdu_vocab))
print("English vocabulary is of {} unique words".format(english_vocab))


Maximum length urdu sentence: 32
Maximum length english sentence: 30
urdu vocabulary is of 2409 unique words
English vocabulary is of 1881 unique words


In [ ]:
max_urdu_len = int(len(max(urd_text_tokenized,key=len)))
max_english_len = int(len(max(eng_text_tokenized,key=len)))

urd_pad_sentence = pad_sequences(urd_text_tokenized, max_urdu_len,
                                 padding = "post")
eng_pad_sentence = pad_sequences(eng_text_tokenized, max_english_len, 
                                 padding = "post")

# Reshape data
urd_pad_sentence = urd_pad_sentence.reshape(*urd_pad_sentence.shape, 1)
eng_pad_sentence = eng_pad_sentence.reshape(*eng_pad_sentence.shape, 1)

In [ ]:
input_sequence = Input(shape=(max_urdu_len,))
embedding = Embedding(input_dim=urdu_vocab, output_dim=128,) (input_sequence)
encoder = LSTM(64, return_sequences=False)                   (embedding)
r_vec = RepeatVector(max_english_len)                        (encoder)
decoder = LSTM(64, return_sequences=True, dropout=0.2)       (r_vec)
logits = TimeDistributed(Dense(english_vocab))               (decoder)


In [ ]:
enc_dec_model = Model(input_sequence, Activation('softmax')  (logits))
enc_dec_model.compile(loss=sparse_categorical_crossentropy,
              optimizer=Adam(1e-3),
              metrics=['accuracy'])
enc_dec_model.summary()

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 32)]              0         
                                                                 
 embedding_2 (Embedding)     (None, 32, 128)           308352    
                                                                 
 lstm_4 (LSTM)               (None, 64)                49408     
                                                                 
 repeat_vector_2 (RepeatVect  (None, 30, 64)           0         
 or)                                                             
                                                                 
 lstm_5 (LSTM)               (None, 30, 64)            33024     
                                                                 
 time_distributed_2 (TimeDis  (None, 30, 1881)         122265    
 tributed)                                                 

In [ ]:
model_results = enc_dec_model.fit(urd_pad_sentence, 
                                  eng_pad_sentence, 
                                  batch_size=15, epochs=1)


117/117 [==============================] - 12s 68ms/step - loss: 3.1204 - accuracy: 0.7820


In [ ]:
def logits_to_sentence(logits, tokenizer):

    index_to_words = {idx: word for word, idx in tokenizer.word_index.items()}
    index_to_words[0] = '<empty>' 

    return ' '.join([index_to_words[prediction] for prediction in np.argmax(
        logits, 1)])

index = 209
print("The english sentence is: {}".format(english_sentences[index]))
print("The urdu sentence is: {}".format(urdu_sentences[index]))
print('The predicted sentence is :')
print(logits_to_sentence(enc_dec_model.predict(urd_pad_sentence[
                                    index:index+1])[0], eng_text_tokenizer))

The english sentence is: I bought this TV on the installment plan.
The urdu sentence is: میں نے یہ ٹی وی انسٹالمنٹ میں خریدا ہے۔
The predicted sentence is :
<empty> <empty> <empty> <empty> <empty> <empty> <empty> <empty> <empty> <empty> <empty> <empty> <empty> <empty> <empty> <empty> <empty> <empty> <empty> <empty> <empty> <empty> <empty> <empty> <empty> <empty> <empty> <empty> <empty> <empty>


In [ ]:
print(logits_to_sentence(enc_dec_model.predict(urd_pad_sentence[
                                    index:index+1])[0], eng_text_tokenizer))

<empty> <empty> <empty> <empty> <empty> <empty> <empty> <empty> <empty> <empty> <empty> <empty> <empty> <empty> <empty> <empty> <empty> <empty> <empty> <empty> <empty> <empty> <empty> <empty> <empty> <empty> <empty> <empty> <empty> <empty>


In [ ]:
input_sentence = 'I bought this TV on the installment plan'
def translate(input_sentence):
  output_sentence = enc_dec_model(input_sentence, eng_text_tokenizer )
  return output_sentence
  print(output_sentence)

In [ ]:
input 
input_sentence_token, input_text_tokenizer = tokenize(input_sentence)